In [1]:
# import libraries, Classes & Dicts
import pandas as pd
import psycopg2
from ClassesAndDicts import * # please run ClassesAndDicts.py first

In [9]:
# create and connect to the database git
try:
    connection = psycopg2.connect(
        dbname = "postgres",
        user = "postgres",
        password = "0324",
        host = "localhost",
        port = "4321" # note my port is 4321, please change if yours is 5432
    )
    connection.autocommit = True
    
    cursor = connection.cursor()
    
    cursor.execute("CREATE DATABASE git;")
    print(cursor.statusmessage)    
except psycopg2.Error as error:
    print("Warning!", error)
finally:
    if connection is not None:
        cursor.close()
        connection.close()

Warning! database "git" already exists



## gitIssues

In [29]:
# Read data gitIssues.csv
gitIssues_df = pd.read_csv('C:\\Users\\Dell\\Desktop\\DMS\\gitIssues.csv',sep=',').iloc[:, 1:]

In [30]:
# create gitIssues schema and tables
try:
    connection = psycopg2.connect(
        **connection_params
    )
    
    cursor = connection.cursor()
    
    # Create gitIssues schema
    schema = Schema("gitIssues")
    schema.drop_schema_cascade(cursor)
    schema.create_schema(cursor)
    
    # create and insert issues tables
    tables = {}
    tablenames = "issues", "users", "comments", "comment_user"
    for tablename in tablenames:
        tables[tablename] = schema.new_table(gitIssues_df, *git_issues_data_dict[tablename], tablename)
    
    tables["users"].rename_fields("user_id,user_name")
    for name, table in tables.items():
        table.create_table(cursor)
        table.insert_into(cursor)
    connection.commit()
except psycopg2.Error as error:
    print("Warning!", error)
finally:
    if connection is not None:
        cursor.close()
        connection.close()

DROP SCHEMA 

CREATE SCHEMA 

CREATE TABLE 

CREATE TABLE 

CREATE TABLE 

CREATE TABLE 



In [31]:
# Create 3 new tables(users_ issues_ comment_user_) based on the current 3 tables (users issues comment_user)
# Then DROP the current 3 tables (users issues comment_user)
# THis section is for delete duplicated rows
try:
    connection = psycopg2.connect(
        **connection_params
    )
    
    cursor = connection.cursor()
    
    cursor.execute("""
    CREATE TABLE gitIssues.issues_ as select distinct * from gitIssues.issues;
    CREATE TABLE gitIssues.users_ as select user_name, user_id, count(*) as user_count from gitIssues.users group by user_name, user_id;
    CREATE TABLE gitIssues.comment_user_ as select distinct * from gitIssues.comment_user;
    """)
    print(cursor.statusmessage, '\n')
    tables["issues"].drop_table_cascade(cursor)
    tables["users"].drop_table_cascade(cursor)
    tables["comment_user"].drop_table_cascade(cursor)
    connection.commit()
except psycopg2.Error as error:
    print("Warning!", error)
finally:
    if connection is not None:
        cursor.close()
        connection.close()

SELECT 4242 

DROP TABLE 

DROP TABLE 

DROP TABLE 



In [32]:
# add constraints (primary_key, foreign_key )
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()
    cursor.execute(""" 
    
       ALTER TABLE gitIssues.issues_ ADD PRIMARY KEY (title,created_at); --For table issues_ , title & created_at are the composite primary key
       ALTER TABLE gitIssues.comments ADD PRIMARY KEY (comment_id);
       ALTER TABLE gitIssues.users_ ADD PRIMARY KEY (user_id);
       ALTER TABLE gitIssues.comment_user_ ADD PRIMARY KEY (comment_user_id);

    -- link issues_ with users_ by foreign key user_id 
       ALTER TABLE gitIssues.issues_ ADD CONSTRAINT issues_fk1 FOREIGN KEY(user_id) REFERENCES gitIssues.users_(user_id);
    -- link comments with users_ by foreign key user_id 
       ALTER TABLE gitIssues.comments ADD CONSTRAINT comments_fk1 FOREIGN KEY(user_id) REFERENCES gitIssues.users_(user_id);
    -- link comments with issues_ by foreign key (title,created_at)
       ALTER TABLE gitIssues.comments ADD CONSTRAINT comments_fk2 FOREIGN KEY(title,created_at) REFERENCES gitIssues.issues_(title,created_at);
    -- link comments with comment_user_ by foreign key comment_user_id
       ALTER TABLE gitIssues.comments ADD CONSTRAINT comments_fk3 FOREIGN KEY(comment_user_id) REFERENCES gitIssues.comment_user_(comment_user_id);
    
    """)
    print(cursor.statusmessage, '\n')
    connection.commit()
except psycopg2.Error as error:
    print("Warning!", error)
finally:
    if connection is not None:
        cursor.close()
        connection.close()

ALTER TABLE 



## gitData

In [18]:
# Read data gitData.csv
gitData_df = pd.read_csv('C:\\Users\\Dell\\Desktop\\DMS\\gitData.csv',sep=',').iloc[:, 1:]

In [20]:
# create gitData schema and tables
try:
    connection = psycopg2.connect(
        **connection_params
    )
    
    cursor = connection.cursor()
    
    # Create gitData schema
    schema = Schema("gitData")
    schema.create_schema(cursor)
    
    # create and insert gitData tables
    tables = {}
    tablenames = "commits", "files"
    for tablename in tablenames:
        tables[tablename] = schema.new_table(gitData_df, *git_data_data_dict[tablename], tablename)
    
    for name, table in tables.items():
        table.create_table(cursor)
        table.insert_into(cursor)
    connection.commit()
except psycopg2.Error as error:
    print("Warning!", error)
finally:
    if connection is not None:
        cursor.close()
        connection.close()

CREATE SCHEMA 

CREATE TABLE 

CREATE TABLE 



In [25]:
# Create 2 new tables( commits_ , files_ ) based on the current table ( commits files)
# Then DROP the current 2 tables ( commits files )
# THis section is for delete duplicated rows
# Also create a new table named author (with commits_count and first_author_date)
try:
    connection = psycopg2.connect(
        **connection_params
    )
    
    cursor = connection.cursor()
    
    cursor.execute("""
    CREATE TABLE gitData.commits_ as select distinct * from gitData.commits;
    CREATE TABLE gitData.files_ as select distinct * from gitData.files where nloc!='NaN';

    CREATE TABLE gitData.author as 
    (select author_name, project_name, count(*) as commits_count, min(author_date) as first_author_date 
    from gitData.commits_ group by author_name,project_name order by count(*) desc);

    """)
    print(cursor.statusmessage, '\n')
    tables["commits"].drop_table_cascade(cursor)
    tables["files"].drop_table_cascade(cursor)
    connection.commit()
except psycopg2.Error as error:
    print("Warning!", error)
finally:
    if connection is not None:
        cursor.close()
        connection.close()

SELECT 28578 

DROP TABLE 



In [28]:
# add constraints (primary_key, foreign_key )
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()
    cursor.execute(""" 
       ALTER TABLE gitData.commits_ ADD PRIMARY KEY (hash);      
       ALTER TABLE gitData.files_ ADD PRIMARY KEY (hash,old_path,new_path); --For table files_, hash-old_path-new_path are the composite primary key
       ALTER TABLE gitData.author ADD PRIMARY KEY (author_name,project_name);

    -- link commits_ with author by foreign key (author_name,project_name)
       ALTER TABLE gitData.commits_ ADD CONSTRAINT commits_fk1 FOREIGN KEY(author_name,project_name) REFERENCES gitData.author(author_name,project_name);
    -- link files_ with commits_ by foreign key hash
       ALTER TABLE gitData.files_ ADD CONSTRAINT files_fk1 FOREIGN KEY(hash) REFERENCES gitData.commits_(hash);
    
    """)
    print(cursor.statusmessage, '\n')
    connection.commit()
except psycopg2.Error as error:
    print("Warning!", error)
finally:
    if connection is not None:
        cursor.close()
        connection.close()

ALTER TABLE 

